In [ ]:
i=0
for k in samples.keys():
    if k=="true":
        continue
    fake=samples[k]
    fig,ax=plt.subplots(1,3,figsize=plot.fig_size3)

    for i in range(3):
        true=samples["true"].reshape(-1,30,3).clone()

        m_up=float(k.split("=")[-1].split("$")[0])+0.02
        true=true[mass(true[:,:,:])<m_up]
        m_down=float(k.split("=")[-1].split("$")[0])-0.02
        true=true[mass(true[:,:,:])>m_down]

        _,b,_=ax[i].hist(true.reshape(-1,3)[:,i].numpy(),bins=30,alpha=0.3,label="Ground Truth ${:.2f}< m^{}<{:.2f}$".format(m_down,"{rel}",m_up),color=plot.cmap[1],density=1)
        ax[i].set_ylabel("Counts (Normalised)")
        # ax2 =ax[i].twinx()
        ax[i].ticklabel_format(axis="y", style="scientific", scilimits=(-3, 3), useMathText=True)
        # ax2.ticklabel_format(axis="y", style="scientific", scilimits=(-3, 3), useMathText=True)
        ax[i].hist(fake.reshape(-1,3)[:,i].numpy(),bins=b,alpha=1,label="Generated",histtype="step",lw=4,density=1)
        # ax2.set_ylabel("Counts Generated")
        handles = [mpatches.Patch(color=sns.color_palette()[z], label=label)  for z, label in enumerate(["Ground Truth ${:.2f}< m^{}<{:.2f}$".format(m_down,"{rel}",m_up),"Generated "+k])]
        ax[-1].legend(handles=handles, fontsize=15,loc="upper right")
        ax[i].set_xlabel(["$\eta^{rel}$","$\phi^{rel}$","$p_T^{rel}$"][i])
    # plt.suptitle("Particle Feature Distribution for Different Mass Conditionings", fontsize=28, fontweight="bold")
    plt.tight_layout()
    plt.savefig("plots/feature_mass_conditioning_{}.pdf".format(k.split("=")[-1].split("$")[0]),format="pdf")
    plt.show()